In [4]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
from scipy import misc

import imageio
from PIL import Image
from sklearn.preprocessing import OneHotEncoder
# import pillow
import categories
import helper_function

import importlib

In [5]:
"""
输入：
含有image_id 和 category 的数组 和 坐标
"""
all_columns = np.array( ['neckline_left', 'neckline_right',\
       'center_front', 'shoulder_left', 'shoulder_right', 'armpit_left',\
       'armpit_right', 'waistline_left', 'waistline_right', 'cuff_left_in',\
       'cuff_left_out', 'cuff_right_in', 'cuff_right_out', 'top_hem_left',\
       'top_hem_right', 'waistband_left', 'waistband_right', 'hemline_left',\
       'hemline_right', 'crotch', 'bottom_left_in', 'bottom_left_out',\
       'bottom_right_in', 'bottom_right_out'] )

blouse_columns = np.array( [ 'neckline_left', 'neckline_right',\
       'center_front', 'shoulder_left', 'shoulder_right', 'armpit_left',\
       'armpit_right',  'cuff_left_in', 'cuff_left_out', 'cuff_right_in', 'cuff_right_out', 'top_hem_left',\
       'top_hem_right'] )

outwear_columns =  np.array( ['neckline_left', 'neckline_right',\
     'shoulder_left', 'shoulder_right', 'armpit_left',\
       'armpit_right', 'waistline_left', 'waistline_right', 'cuff_left_in',\
       'cuff_left_out', 'cuff_right_in', 'cuff_right_out', 'top_hem_left',\
       'top_hem_right'] )

trousers_columns = np.array( [ 'waistband_left', 'waistband_right',  'crotch', 'bottom_left_in', 'bottom_left_out',\
       'bottom_right_in', 'bottom_right_out'] )

skirt_columns = np.array( [ 'waistband_left', 'waistband_right',  'hemline_left', 'hemline_right'] )

dress_columns = np.array( ['neckline_left', 'neckline_right',\
       'center_front', 'shoulder_left', 'shoulder_right', 'armpit_left',\
       'armpit_right', 'waistline_left', 'waistline_right', 'cuff_left_in',\
       'cuff_left_out', 'cuff_right_in', 'cuff_right_out', 'hemline_left',\
       'hemline_right'] )

print(blouse_columns.shape,outwear_columns.shape,trousers_columns.shape,skirt_columns.shape,dress_columns.shape)

(13,) (14,) (7,) (4,) (15,)


In [8]:
"""
convert predication on test 


"""


input_dir = './output/tmp/'
output_dir = "output/"


df_size = pd.read_csv("./test_pad/test_size.csv")



all_data = []
output_file = output_dir + "result_cpm.csv"

cates = ["blouse" ,"outwear","trousers","skirt","dress" ]
cates = ["blouse", "skirt","outwear","dress" ,"trousers"]
for cate in cates:
    df_train = pd.read_csv("./train_pad/Annotations/train_"+cate+"_coord.csv")
    
    input_file =input_dir+ "cpm_"+cate+"_512.csv"
    df_data_coord = pd.read_csv(input_file)
    output_data = df_data_coord.iloc[:,:2]
    output_data.columns = ['image_id', 'image_category']


# for cate in origin_cols:
    
#     input_file =input_dir+ "test_"+cate+"_coord.csv"
#     data_coord = pd.read_csv(input_file)
    im_size=512

    #处理负数 以及处理Padding
    for idx,row in df_data_coord.iterrows():
        row_size = df_size.loc[df_size['image_id'] == row['image_id']]
        height = row_size.height.values[0]
        width = row_size.width.values[0]
        #将预测出负数的值变为该点的均值：
#         x_start=2
#         for i in np.arange(x_start,row.shape[0] , 3):
#             if row[i+2] != -1:
# #                 if row[i+2] == 0:
# #                     row[i+2] = 1
#                 if row[i]<=0:
#                     row[i] = int(df_train.iloc[:,i].mean())
#                 if row[i+1]<=0:
#                     row[i+1] =int(df_train.iloc[:,i+1].mean())
#                 df_data_coord.loc[idx,:] = row

        #减去padding后的坐标值
        if height<im_size or width<im_size:
            wid_diff = im_size - width
            height_diff = im_size - height
            left = int(wid_diff/2)
            up = int(height_diff/2)
            x_start=2
            y_start = 3
            x_index = np.arange(x_start , row.shape[0] , 3)
            y_index= np.arange(y_start , row.shape[0] , 3)
            row[x_index] -=left
            row[y_index] -=up
            df_data_coord.loc[idx,:] = row
    begin_col=2
    interval=3
    cate_cols = categories.get_columns(cate)

    for col in all_columns:
        if col not in cate_cols:
            output_data[col] = '-1_-1_-1'
        else:
            coords = df_data_coord.iloc[:,begin_col:begin_col+interval].as_matrix().astype(int)
            vstr = np.vectorize(str)
            coords = vstr(coords)
            output_data[col] = ["_".join(item) for item in coords]
            begin_col+=interval
    all_data.append(output_data)   
output_all = pd.concat(all_data,ignore_index=True)
output_all.to_csv(output_file , index  = False)
# # #选择适合的 衣服类型
# cate_cols = categories.get_columns("skirt")
# #if blouse
# begin_col=2
# interval=3
# for col in all_columns:
#     if col not in cate_cols:
#         output_data[col] = '-1_-1_-1'
#     else:
        
#         coords = df_data_coord.iloc[:,cnt:cnt+interval].as_matrix()

#         vstr = np.vectorize(str)
#         coords = vstr(coords)
        
#         output_data[col] = ["_".join(item) for item in coords]
#         print(output_data[col])
#         begin_col+=interval

# all_data.append(output_data)
    


In [ ]:
mean = pd.read_csv("test/result_mean.csv")
helper_function.show_im_lms(mean,9010,1 , "test/")

In [ ]:
a = pd.read_csv("test/result_cpm.csv")
helper_function.show_im_lms(a,1000,1 , "test/")
